In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix)
import spacy
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
class DataPreprocessing:
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')

        self.category_mapping = {
            'Fields of mathematics': 'Mathematics',
            'Mathematics-related lists': 'Mathematics',
            'Branches of biology': 'Biology',
            'Organisms': 'Biology',
            'Biologists': 'Biology',
            'Biology-related lists': 'Biology',
            'Geographers': 'Geography',
            'Geography-related lists': 'Geography',
            'History by ethnic group': 'History',
            'History by period': 'History',
            'Fields of history': 'History',
            'Historiography': 'History',
            'Branches of science': 'Science',
            'Scientific disciplines': 'Science',
            'Scientists': 'Science',
            'Science in society': 'Science'
        }
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        self.tokenizer = RegexpTokenizer(r'\w+')

    def map_categories(self, df):
        #Replace categories in the DataFrame using the mapping.
        df['Category'] = df['Category'].replace(self.category_mapping)
        return df

    def get_unique_categories(self, df):
        #Return unique categories after mapping.
        return df['Category'].unique()

    def remove_stop_words(self, text):
        #Remove stop words from the given text.
        text = text.lower()
        words = text.split()
        filtered_words = [word for word in words if word not in self.stop_words]
        return ' '.join(filtered_words)

    def process_content(self, df):
        #Apply stop word removal to the Content column.
        df['Content'] = df['Content'].apply(self.remove_stop_words)
        return df

    def metadata_fix(self, text):
        #Clean HTML tags and unwanted characters from the text.
        soup = BeautifulSoup(text, "html.parser")
        cleaned_text = soup.get_text()
        cleaned_text = re.sub(r'http\S+|www\S+|https\S+', '', cleaned_text)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_text)
        return cleaned_text

    def fix_metadata(self, df):
        #Apply metadata cleaning to the Content column.
        df['Content'] = df['Content'].apply(self.metadata_fix)
        return df

    def lemmatize_words(self, text):
        #Lemmatize words in the given text.
        words = word_tokenize(text)
        lemmatized_words = [self.lemmatizer.lemmatize(word) for word in words]
        return ' '.join(lemmatized_words)

    def lemmatize_content(self, df):
        #Apply lemmatization to the Content column
        df['Content'] = df['Content'].apply(self.lemmatize_words)
        return df

    def tokenize_content_spacy(self, df):
        #Tokenize content using spaCy.
        df['Content'] = df['Content'].apply(lambda x: [token.text for token in self.nlp(x)])
        return df

